# Doing some Imports

In [ ]:
import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Import necessary items from Keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
from keras.layers import concatenate
from keras.losses import binary_crossentropy
from keras.layers import Input
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

## Create your model

In [ ]:
def create_model(input_shape):
    # TODO: Create the Model
    num_filters = 32
    num_classes = 1
    initializer = tf.keras.initializers.GlorotUniform

    #input layer
    inputs = Input(input_shape)
    #Batch normalization to normalize inputs
    X = BatchNormalization()(inputs)
    
    #2 Conv layers with relu activation and number of filter = num_filters
    X = Conv2D(filters = num_filters, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = num_filters, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    ConvBlockOut1 = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2), strides=(2,2))(ConvBlockOut1)
    
    #3 Conv layers with relu activation and number of filter = num_filters * 2, and dropout with prob = 0.2
    X = Conv2D(filters = num_filters * 2, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    X = Conv2D(filters = num_filters * 2, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    X = Conv2D(filters = num_filters * 2, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = Activation('relu')(X)
    ConvBlockOut2 = Dropout(0.2)(X)
    X = MaxPooling2D(pool_size=(2,2), strides=(2,2))(ConvBlockOut2)
    
    #2 Conv layers with relu activation and number of filter = num_filters * 4, and dropout with prob = 0.2
    X = Conv2D(filters = num_filters * 4, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    X = Conv2D(filters = num_filters * 4, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = Activation('relu')(X)
    ConvBlockOut3 = Dropout(0.2)(X)
    X = MaxPooling2D(pool_size=(2,2), strides=(2,2))(ConvBlockOut3)

    X = Conv2D(filters = num_filters * 8, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = Conv2D(filters = num_filters * 8, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = Conv2D(filters = num_filters * 8, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)

    # Upsampling phase
    X = UpSampling2D(size=(2, 2))(X)
    merge1 = concatenate([ConvBlockOut3, X], axis=-1)
    X = Conv2DTranspose(filters = num_filters * 4, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(merge1)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    X = Conv2DTranspose(filters = num_filters * 4, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)

    X = UpSampling2D(size=(2, 2))(X)
    merge2 = concatenate([ConvBlockOut2, X], axis=-1)
    X = Conv2DTranspose(filters = num_filters * 2, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(merge2)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    X = Conv2DTranspose(filters = num_filters * 2, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    X = Conv2DTranspose(filters = num_filters * 2, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)

    X = UpSampling2D(size=(2, 2))(X)
    merge3 = concatenate([ConvBlockOut1, X], axis=-1)
    X = Conv2DTranspose(filters = num_filters, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(merge3)
    X = Activation('relu')(X)
    X = Conv2DTranspose(filters = num_filters, kernel_size = 3, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = Activation('relu')(X)

    #output of the ConvNet to mirror number of classes with kernel size of 1 to reduce number of channels but keep image width and height
    outputs = Conv2D(num_classes, 1, padding='same',activation = 'sigmoid')(X)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model

In [ ]:
model = create_model((64, 64, 3))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 3)   12          ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 32)   896         ['batch_normalization[0][0]']    
                                                                                                  
 activation (Activation)        (None, 64, 64, 32)   0           ['conv2d[0][0]']             

## Load the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load training images
train_images = pickle.load(open("/content/drive/MyDrive/DLProject/full_CNN_train.p", "rb" ))

# Load image labels
labels = pickle.load(open("/content/drive/MyDrive/DLProject/full_CNN_labels.p", "rb" ))

# Make into arrays as the neural network wants these
train_images = np.array(train_images)
labels = np.array(labels)
labels = (labels>128).astype('uint8')

## Preprocess the data

In [ ]:
# TODO: Preprocess the images
X = train_images.shape  # (12764, 80, 160, 3)
Y = labels.shape  # (12764, 80, 160, 1)

X[1:]

(80, 160, 3)

## Create the train and test splits

In [ ]:
# Shuffle images along with their labels, then split into training/validation sets
train_images, labels = shuffle(train_images, labels)
# Test size may be 10% or 20%
X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.1)

## Train your model

In [ ]:
# TODO: Train your model
datagen = ImageDataGenerator(channel_shift_range=30, brightness_range=[0, 2])
datagen.fit(X_train)
input_shape = X_train.shape[1:]
# Create the neural network
model = create_model(input_shape)

step = tf.Variable(0, trainable=False)
boundaries = [900, 1600]
values = [0.001, 0.0005, 0.0004]
learning_rate_fn = PiecewiseConstantDecay(boundaries, values)

# Later, whenever we perform an optimization step, we pass in the step.
learning_rate = learning_rate_fn(step)
opt = Adam(learning_rate=learning_rate)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.fit_generator(datagen.flow(X_train, y_train, batch_size=64), steps_per_epoch=len(X_train) / 64, epochs=19, verbose=1, validation_data=(X_val, y_val), callbacks=[model_checkpoint_callback])
#model.fit(X_train,y_train,epochs= 5,batch_size=64)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/19
179/179 [==============================] - 109s 592ms/step - loss: 0.2579 - accuracy: 0.9145 - val_loss: 0.0717 - val_accuracy: 0.9760
Epoch 2/19
179/179 [==============================] - 106s 592ms/step - loss: 0.0553 - accuracy: 0.9783 - val_loss: 0.0351 - val_accuracy: 0.9871
Epoch 3/19
179/179 [==============================] - 106s 589ms/step - loss: 0.0391 - accuracy: 0.9848 - val_loss: 0.0279 - val_accuracy: 0.9897
Epoch 4/19
179/179 [==============================] - 106s 590ms/step - loss: 0.0341 - accuracy: 0.9867 - val_loss: 0.0282 - val_accuracy: 0.9900
Epoch 5/19
179/179 [==============================] - 106s 590ms/step - loss: 0.0316 - accuracy: 0.9877 - val_loss: 0.0241 - val_accuracy: 0.9906
Epoch 6/19
179/179 [==============================] - 106s 592ms/step - loss: 0.0295 - accuracy: 0.9884 - val_loss: 0.0231 - val_accuracy: 0.9911
Epoch 7/19
179/179 [==============================] - 106s 591ms/step - loss: 0.0286 - accuracy: 0.9887 - val_loss: 0.0224 -

In [ ]:
model.evaluate(X_val,y_val)

40/40 [==============================] - 4s 95ms/step - loss: 0.0173 - accuracy: 0.9928


[0.017311574891209602, 0.9928343892097473]

## Save your model

In [ ]:
# Save model architecture and weights
model.save('full_CNN_model.h5')

# Show summary of model
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 80, 160, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 80, 160, 3)  12          ['input_4[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 conv2d_33 (Conv2D)             (None, 80, 160, 32)  896         ['batch_normalization_3[0][0]']  
                                                                                            